In [2]:
import os
from PIL import Image
import pytesseract
import PyPDF2
from io import BytesIO

from assets.config import key

pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

def extract_text_from_image(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image, lang='deu') 
    return text

def extract_text_from_pdf_image(pdf_path):
    extracted_text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            if '/XObject' in page['/Resources']:
                xObject = page['/Resources']['/XObject'].get_object()
                for obj in xObject:
                    if xObject[obj]['/Subtype'] == '/Image':
                        size = (xObject[obj]['/Width'], xObject[obj]['/Height'])
                        data = xObject[obj]._data
                        if xObject[obj]['/ColorSpace'] == '/DeviceRGB':
                            mode = "RGB"
                        else:
                            mode = "P"

                        if '/Filter' in xObject[obj]:
                            if xObject[obj]['/Filter'] == '/DCTDecode':
                                img = Image.open(BytesIO(data))
                                text = pytesseract.image_to_string(img, lang='deu')
                                extracted_text += text
                            elif xObject[obj]['/Filter'] == '/FlateDecode':
                                img = Image.frombytes(mode, size, data)
                                text = pytesseract.image_to_string(img, lang='deu')
                                extracted_text += text
    return extracted_text

folder_path = '../data'

extracted_texts = []

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    print(file_path)
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        text = extract_text_from_image(file_path)
        extracted_texts.append(text)
    elif filename.lower().endswith('.pdf'):
        text = extract_text_from_pdf_image(file_path)
        extracted_texts.append(text)


../data\AB1.jpg
../data\AB2.png
../data\AB3.pdf
../data\AB4.png


In [5]:
import openai

openai.api_key = key

prompt = (
    "Bitte analysiere den folgenden Arztbrief und extrahiere die relevanten Informationen zu den vorgegebenen Stichwörtern. "
    "Wenn du Informationen zu einem Stichwort findest, übertrage die Textstellen aus dem Arztbrief"
    "Wenn keine Information zu einem Stichwort gefunden wird, gib 'NA' zurück. "
    "Hier ist der Arztbrief:\n\n"
    f"{extracted_texts[1]}\n\n"
    "Bitte extrahiere die Informationen zu den folgenden Stichwörtern:\n\n"
    "- 'Adressat': Empfänger des Arztbriefes\n"
    "- 'Versanddatum des Arztbriefes': Datum des Versands\n"
    "- 'Vorstellungsdatum Patient': Datum der Vorstellung des Patienten\n"
    "- 'Aufenthaltsdauer Patient': Dauer des Aufenthalts\n"
    "- 'Patientenname': Name des Patienten\n"
    "- 'Überweisungsgrund': Grund für die Überweisung\n"
    "- 'Krankengeschichte': Beschreibung der Krankengeschichte\n"
    "- 'Körperliche Untersuchung': Durchführung und Ergebnisse der körperlichen Untersuchung\n"
    "- 'Klinische Untersuchung': Durchführung und Ergebnisse der klinischen Untersuchung\n"
    "- 'Diagnostik': Durchgeführte Diagnostik\n"
    "- 'Invasive Eingriffe': Durchgeführte invasive Eingriffe\n"
    "- 'Operationen': Durchgeführte Operationen\n"
    "- 'Epikrise': Abschließende Betrachtung\n"
    "- 'Therapieempfehlung': Empfehlung für die weitere Therapie\n"
    "- 'Anhang': Vorhandensein eines Anhangs\n\n"
    "Bitte strukturiere die Antworten wie folgt und ersetze 'passende Textstellen' mit den unveränderten Textstellen aus dem Arztbrief:\n\n"
    "[\n"
    "    'Adressat', 'passende Textstellen',\n"
    "    'Versanddatum des Arztbriefes', 'passende Textstellen',\n"
    "    'Vorstellungsdatum Patient', 'passende Textstellen',\n"
    "    'Aufenthaltsdauer Patient', 'passende Textstellen',\n"
    "    'Patientenname', 'passende Textstellen',\n"
    "    'Überweisungsgrund', 'passende Textstellen',\n"
    "    'Krankengeschichte', 'passende Textstellen',\n"
    "    'Körperliche Untersuchung', 'passende Textstellen',\n"
    "    'Klinische Untersuchung', 'passende Textstellen',\n"
    "    'Diagnostik', 'passende Textstellen',\n"
    "    'Invasive Eingriffe', 'passende Textstellen',\n"
    "    'Operationen', 'passende Textstellen',\n"
    "    'Epikrise', 'passende Textstellen',\n"
    "    'Therapieempfehlung', 'passende Textstellen',\n"
    "    'Anhang', 'passende Textstellen'\n"
    "]"
)

response = openai.ChatCompletion.create(
  model='gpt-3.5-turbo-0125',
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)

# Extrahieren Sie die Antwort
antwort = response.choices[0].message['content']

import pandas as pd

def convert_categorization_to_dataframe(result_string):
    if not isinstance(result_string, str):
        raise ValueError("Eingabedaten sind nicht vom Typ String")

    if not result_string.startswith("[\n    '") or not result_string.endswith("'\n]"):
        raise ValueError("Eingabedaten haben nicht das erwartete Format. Das Modell hat einen Fehler beim \
                         strukturieren der Ergebnisse gemacht.")

    expected_keywords = [
        'Adressat',
        'Versanddatum des Arztbriefes',
        'Vorstellungsdatum Patient',
        'Aufenthaltsdauer Patient',
        'Patientenname',
        'Überweisungsgrund',
        'Krankengeschichte',
        'Körperliche Untersuchung',
        'Klinische Untersuchung',
        'Diagnostik',
        'Invasive Eingriffe',
        'Operationen',
        'Epikrise',
        'Therapieempfehlung',
        'Anhang'
    ]

    try:
        # Entferne die umgebenden Zeichen "[\n    '", "'\n]" und unnötige Whitespaces.
        clean_string = result_string.strip()[4:-3].strip()

        # Teile den String anhand von "',\n    '" um die einzelnen Paare zu erhalten.
        pairs = clean_string.split("',\n    '")

        # Erstelle eine Liste von Dictionaries für jedes Schlüssel-Wert-Paar.
        data = []
        for pair in pairs:
            # Entferne die Anführungszeichen von jedem Schlüssel-Wert-Paar.
            key_value = pair.strip("'").split("', '")
            if len(key_value) != 2:
                raise ValueError(f"Ein Paar hat nicht das erwartete 'key', 'value' Format: {pair}")
            # Füge das bereinigte Paar den Daten hinzu.
            data.append({'Stichwort': key_value[0], 'Textstelle': key_value[1]})

        # Erstelle einen DataFrame aus der Liste von Dictionaries.
        dataframe = pd.DataFrame(data)

        # Überprüfe, ob alle erwarteten Stichwörter im DataFrame enthalten sind.
        missing_keywords = set(expected_keywords) - set(dataframe['Stichwort'])
        if missing_keywords:
            raise ValueError(f"Die Stichwörter entsprechend nicht den Vorgaben. Das Modell hat einen \
                             Fehler beim strukturieren der Ergebnisse gemacht: {missing_keywords}")

        return dataframe
    except Exception as e:
        # Gib eine aussagekräftige Fehlermeldung aus, wenn ein Fehler auftritt.
        raise ValueError(f"Ein Fehler ist beim Verarbeiten der Eingabedaten aufgetreten: {e}")
    
    # Rufen Sie die Funktion auf und drucken Sie den DataFrame.
try:
    df = convert_categorization_to_dataframe(antwort)
    print(df)
except ValueError as ve:
    print(ve)


                       Stichwort  \
0                       Adressat   
1   Versanddatum des Arztbriefes   
2      Vorstellungsdatum Patient   
3       Aufenthaltsdauer Patient   
4                  Patientenname   
5              Überweisungsgrund   
6              Krankengeschichte   
7       Körperliche Untersuchung   
8         Klinische Untersuchung   
9                     Diagnostik   
10            Invasive Eingriffe   
11                   Operationen   
12                      Epikrise   
13            Therapieempfehlung   
14                        Anhang   

                                           Textstelle  
0                                   Hr. Karsten Weber  
1                                                  NA  
2                                          20.07.2016  
3                                                  NA  
4                                                  NA  
5                             V.a. zerebraler Infarkt  
6   Hängen des linken Mundwinke

In [ ]:
import openai

openai.api_key = key

prompt = (
    "Bitte analysiere den folgenden Arztbrief und beantworte die Fragen zu den vorgegebenen Stichwörtern. "
    "Wähle zwischen 'ja', 'nein' und 'unklar', wenn die Informationen aus dem Text nicht eindeutig sind. "
    "Gib eine kurze Einschätzung ab, wenn nach dem Gesamtzustand des Patienten gefragt wird. "
    "Hier ist der Arztbrief:\n\n"
    f"{extracted_texts[1]}\n\n"
    "Bitte beantworte die folgenden Fragen basierend auf den Informationen im Arztbrief:\n\n"
    "- 'Weitere Behandlung notwendig': Ist eine weitere Behandlung notwendig? [ja/nein/unklar]\n"
    "- 'Komplikationen aufgetreten': Sind Komplikationen bei der Behandlung aufgetreten? [ja/nein/unklar]\n"
    "- 'Gesamtzustand des Patienten': Wie ist der Gesamtzustand des Patienten? [Kurze Einschätzung]\n\n"
    "Bitte strukturiere die Antworten wie folgt:\n\n"
    "[\n"
    "    'Weitere Behandlung notwendig', 'ja/nein/unklar',\n"
    "    'Komplikationen aufgetreten', 'ja/nein/unklar',\n"
    "    'Gesamtzustand des Patienten', 'Kurze Einschätzung'\n"
    "]"
)

response = openai.ChatCompletion.create(
  model='gpt-4-0125-preview',
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)

# Extrahieren Sie die Antwort
antwort = response.choices[0].message['content']


In [ ]:
antwort

"[\n    'Weitere Behandlung notwendig', 'ja',\n    'Komplikationen aufgetreten', 'nein',\n    'Gesamtzustand des Patienten', 'Der Patient präsentiert sich in einem Zustand, der aufmerksame medizinische Beobachtung und weitere Diagnostik erfordert, insbesondere wegen des Verdachts auf einen zerebralen Infarkt, manifeste Schwäche und Sensibilitätsverlust der linken Körperhälfte. Der allgemein gute Gesundheitszustand und die Abwesenheit akuter Notfälle im EKG und CT kontrastieren mit neurologischen Symptomen, die eine weiterführende Behandlung notwendig machen.'\n]"

In [ ]:
import pandas as pd

def convert_outcomes_to_dataframe(result_string):
    if not isinstance(result_string, str):
        raise ValueError("Eingabedaten sind nicht vom Typ String")

    if not result_string.startswith("[\n    '") or not result_string.endswith("'\n]"):
        raise ValueError("Eingabedaten haben nicht das erwartete Format. Das Modell hat einen Fehler beim \
                         Strukturieren der Ergebnisse gemacht.")

    expected_keywords = [
        'Weitere Behandlung notwendig',
        'Komplikationen aufgetreten',
        'Gesamtzustand des Patienten'
    ]

    try:
        # Entferne die umgebenden Zeichen "[\n    '", "'\n]" und unnötige Whitespaces.
        clean_string = result_string.strip()[4:-3].strip()

        # Teile den String anhand von "',\n    '" um die einzelnen Paare zu erhalten.
        pairs = clean_string.split("',\n    '")

        # Erstelle eine Liste von Dictionaries für jedes Schlüssel-Wert-Paar.
        data = []
        for pair in pairs:
            # Entferne die Anführungszeichen von jedem Schlüssel-Wert-Paar.
            key_value = pair.strip("'").split("', '")
            if len(key_value) != 2:
                raise ValueError(f"Ein Paar hat nicht das erwartete 'key', 'value' Format: {pair}")
            # Füge das bereinigte Paar den Daten hinzu.
            data.append({'Stichwort': key_value[0], 'Modellvorhersage': key_value[1]})

        # Erstelle einen DataFrame aus der Liste von Dictionaries.
        dataframe = pd.DataFrame(data)

        # Überprüfe, ob alle erwarteten Stichwörter im DataFrame enthalten sind.
        missing_keywords = set(expected_keywords) - set(dataframe['Stichwort'])
        if missing_keywords:
            raise ValueError(f"Fehlende Stichwörter im DataFrame: {missing_keywords}")

        return dataframe
    except Exception as e:
        # Gib eine aussagekräftige Fehlermeldung aus, wenn ein Fehler auftritt.
        raise ValueError(f"Ein Fehler ist beim Verarbeiten der Eingabedaten aufgetreten: {e}")

In [ ]:
try:
    df = convert_outcomes_to_dataframe(antwort)
    print(df)
except ValueError as ve:
    print(ve)

                      Stichwort  \
0  Weitere Behandlung notwendig   
1    Komplikationen aufgetreten   
2   Gesamtzustand des Patienten   

                                    Modellvorhersage  
0                                                 ja  
1                                               nein  
2  Der Patient präsentiert sich in einem Zustand,...  


In [ ]:
import openai

openai.api_key = key

prompt = (
    "Bitte fasse die genaue Diagnose und den weiteren Therapieverlauf aus diesem Arztbrief in einfacher, \
        freundliche und sachlicher Sprache auf maximal 300 Zeichen zusammen"
    "Beginne mit Patient*in ..."
    f"{extracted_texts[1]}\n\n"
)

response = openai.ChatCompletion.create(
  model='gpt-4-0125-preview',
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)

# Extrahieren Sie die Antwort
antwort = response.choices[0].message['content']

In [ ]:
print(antwort)

Patient Hr. Karsten Weber erlitt möglicherweise einen zerebralen Infarkt. Zeigte Schwäche und Koordinationsprobleme der linken Seite. Untersuchungen ergaben keine Infarktzeichen im CT. Weitere Details und Behandlungsverlauf werden nach Laborergebnissen bestimmt.
